In [ ]:
'''
【项目09】  中国姓氏排行研究

作业要求
1、数据清洗、整合
要求：
① 将“data01”、“data02”分别读取，并且合并成一个数据
② 结合“户籍地城市编号”及“中国城市代码对照表”数据，将城市经纬度连接进数据中
③ 分别提取“工作地”中的省、市
提示：
① 可以先读取“data01”、“data02”，然后用pd.concat()来连接数据
② 新建字段“工作地-省”，“工作地-市”，“工作地-区县”,如果数据中“工作地”字段无法提取省和市，则用“未识别”填充单元格
   * 通过查看识别后的单元格，如果字数超过5则为“未识别”

2、查看姓氏“普遍指数”，普遍指数=姓氏人口数量
要求：
① 将数据按照“姓”做统计，找到数量最多的TOP20
② 分别制作图表，查看姓氏TOP20的数量及占比
   * 建议用bokeh出柱状图，并且为联动图表
③ 查看“王”姓的全国分布
   * 这里导出excel高版本文件，用powermap查看，绘制密度图
   * 同时可以尝试用echarts绘制空间柱状图来查看
④ 查看“姬”姓的全国分布
   * 这里导出excel高版本文件，用powermap查看，绘制密度图
   * 同时可以尝试用echarts绘制空间柱状图来查看
提示：
① bokeh中绘制联动图表时用gridplot
② powermap需要office2016的excel才会有，并且必须存储xlsx格式
③ powermap中需要通过在“值”中设置“姓的计数”才能正确显示热力图
④ powermap中可以通过“主题”来调节配色风格 / “平面地图”选项来调整球面可视化或者平面可视化
⑤ echarts绘制图表之前，需要对数据按照“lng”（或者“lat”）汇总，得到同一个地点的该姓氏人数，然后绘图
⑥ ecahrts通过设置以下参数来达到效果：视角远近、点柱大小

3、查看姓氏“奔波指数”，奔波指数=姓氏人均迁徙距离。迁徙距离为户籍地所在地级市与现居住地所在地级市的距离。
要求：
① 根据识别的工作地，通过Geocoding查询到对应坐标
② 选择一个姓氏，计算并查看其姓氏的奔波指数，并计算该姓氏的人均通勤距离
   * 在python中筛选数据之后，qgis内做空间分析
③ 按照起点和终点做计数，汇总同一个迁徙路径的数据
④ 通过echart制作通勤OD图
   * 可以将生成的line文件导出geojson，用空间线性轨迹图来表示
   * 这里线的valye为该迁徙路径的汇总计数   
提示：
① 可以筛选一些好玩的姓氏：汤、朴、廉、何、叶、冉等等 
② 需要对数据的工作地进行筛选，其中“工作地_市”、“工作地_区县”未识别的数据删除掉
③ 导出数据时，尽量columns名用全英文，避免qgis中出现乱码
④ 计算人均通勤距离的时候，需要删除掉户籍地与工作地相同的人（未迁移的人）
⑤ 在官网metrodata.cn的小工具中找到geocoding
⑥ qgis中需要安装插件“LinePlotter”来转线，并在qgis中计算平均通勤距离（需要投影，投影经度带可选48）
⑦ shapefile转geojson时：
   * 注意shapefile文件要投影回wgs84地理坐标系
   * 删除乱码子弹
   * 删除路径距离为0的数据


'''

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

import warnings
warnings.filterwarnings('ignore') 
# 不发出警告

from bokeh.io import output_notebook
output_notebook()
# 导入notebook绘图模块

from bokeh.plotting import figure,show
from bokeh.models import ColumnDataSource
# 导入图表绘制、图标展示模块
# 导入ColumnDataSource模块

Loading BokehJS ...

In [ ]:
'''
1、数据清洗、整合
要求：
① 将“data01”、“data02”分别读取，并且合并成一个数据
② 结合“户籍地城市编号”及“中国城市代码对照表”数据，将城市经纬度连接进数据中
③ 分别提取“工作地”中的省、市
提示：
① 可以先读取“data01”、“data02”，然后用pd.concat()来连接数据
② 新建字段“工作地-省”，“工作地-市”，“工作地-区县”,如果数据中“工作地”字段无法提取省和市，则用“未识别”填充单元格
   * 通过查看识别后的单元格，如果字数超过5则为“未识别”

'''

In [3]:
# 查看数据,数据清洗

import os
os.chdir(r'C:\Users\Administrator\Python数据\网易数据分析\1项目资料\项目09中国姓氏排行研究')
# 创建工作路径

df01 = pd.read_csv('data01.csv',encoding = 'utf-8')
df02 = pd.read_csv('data02.csv',encoding = 'utf-8')
df_city = pd.read_excel('中国行政代码对照表.xlsx')
# 读取数据
print('df01\n',df01.head(3),'\n')
print('df02\n',df02.head(),'\n')
print('df_city\n',df_city.head(3),'\n')

df = pd.concat([df01,df02]) # 默认按照相同列名，接上

print('\ndf切片\n',df[1079100:1079105],'\n')

df['户籍地城市编号'] = pd.to_numeric(df['户籍地城市编号'] , errors='coerce')
# 使用to_numeric转为数值。默认情况下，它不能处理字母型的字符串'pandas'。errors='coerce'可以将无效值强制转换为NaN

df = pd.merge(df,df_city,left_on='户籍地城市编号',right_on = '行政编码')  
df['工作地'] = df['工作地'].str[:15]   # 只提取工作地前15个字符即可
del df['行政编码']
del df['户籍地城市编号']
# 合并数据，添加经纬度字段
# 删除无用字段

print('读取数据共%i条' % len(df))
df.head(10)

df01
    姓 户籍地城市编号            工作地
0  孙  310104  上海市瑞金二路129弄6号
1  潘  370683    山东省日照市北京路中段
2  徐  420106    武汉市珞狮南路517号 

df02
    姓   户籍地城市编号                                                工作地
0  王  210105.0                               沈阳市皇姑区塔湾街83-15号3-2-1
1  朱  330722.0  建设路71号                  疾病二科                  ...
2  刘  342401.0  高新区金桂路1号市场部                                   ...
3  韩  330225.0  象山区丹城西街57号                                    ...
4  刘  110108.0                            北京市海淀区西三环北路25号院红5楼5453号 

df_city
      行政编码    省   市  区/县         lng        lat
0  110100  北京市  北京  北京市  116.395645  39.929986
1  110101  北京市  北京  东城区  116.421885  39.938574
2  110102  北京市  北京  西城区  116.373190  39.934280 


df切片
    姓 户籍地城市编号                                                工作地
0  王  210105                               沈阳市皇姑区塔湾街83-15号3-2-1
1  朱  330722  建设路71号                  疾病二科                  ...
2  刘  342401  高新区金桂路1号市场部                                   ...
3  韩  330225  

,姓,工作地,省,市,区/县,lng,lat
0,孙,上海市瑞金二路129弄6号,上海市,上海,徐汇区,121.446235,31.169152
1,王,上海市松江区新桥镇场南路一弄同,上海市,上海,徐汇区,121.446235,31.169152
2,唐,上海市徐汇区斜土路2897弄5,上海市,上海,徐汇区,121.446235,31.169152
3,沈,有浦东新区海阳路工闷一弄20号,上海市,上海,徐汇区,121.446235,31.169152
4,顾,罗城路700弄21号302室,上海市,上海,徐汇区,121.446235,31.169152
5,王,上海中山南二路960弄1号40,上海市,上海,徐汇区,121.446235,31.169152
6,濮,上海市徐汇区桂林西街85弄68,上海市,上海,徐汇区,121.446235,31.169152
7,张,上海市古美西路753弄31号6,上海市,上海,徐汇区,121.446235,31.169152
8,郭,青岛科技大学化工学院,上海市,上海,徐汇区,121.446235,31.169152
9,陈,广州市东华东路东源新街5号80,上海市,上海,徐汇区,121.446235,31.169152


In [4]:
# 分别提取“工作地”中的省、市、区县
# 新建字段“工作地-省”，“工作地-市”，“工作地-区县”,如果数据中“工作地”字段无法提取省和市，则用“未识别”填充单元格
# *通过查看识别后的单元格，如果字数超过5则为“未识别”

df['工作地_省'] = df['工作地'].str.split('省').str[0]  
# 识别工作地-省

df['工作地_市'] = df['工作地'].str.split('省').str[1].str.split('市').str[0]             
df['工作地_市'][df['工作地_省'].str.len() > 5] = df['工作地_省'].str.split('市').str[0]  
# 识别工作地-市
# 在未识别出省的数据中，可能会有市的信息

df['工作地_区县'] = ''
df['工作地_区县'][(df['工作地_市'].str.len() < 5)&(df['工作地'].str.contains('区'))] = df['工作地'].str.split('市').str[1].str.split('区').str[0] + '区'  
df['工作地_区县'][(df['工作地_市'].str.len() > 5)&(df['工作地'].str.contains('区'))] = df['工作地'].str.split('区').str[0] + '区'  
df['工作地_区县'][(df['工作地_市'].str.len() < 5)&(df['工作地'].str.contains('县'))] = df['工作地'].str.split('市').str[1].str.split('县').str[0] + '县'  
df['工作地_区县'][(df['工作地_市'].str.len() > 5)&(df['工作地'].str.contains('县'))] = df['工作地'].str.split('县').str[0] + '县'
# 识别工作地-区县

df['工作地_省'][df['工作地_省'].str.len() > 5] = '未识别'
df['工作地_市'][df['工作地_市'].str.len() > 5] = '未识别'
df['工作地_区县'][(df['工作地_区县'].str.len() > 5) | (df['工作地_区县'].str.len() < 2)] = '未识别'
# 整理未识别单元格

df.columns = ['姓','工作地','户籍所在地_省','户籍所在地_市','户籍所在地_区县','户籍所在地_lng','户籍所在地_lat',
              '工作地_省','工作地_市','工作地_区县']
print('读取数据共%i条' % len(df))
# 数据整理
df.head(20)

读取数据共1748239条


,姓,工作地,户籍所在地_省,户籍所在地_市,户籍所在地_区县,户籍所在地_lng,户籍所在地_lat,工作地_省,工作地_市,工作地_区县
0,孙,上海市瑞金二路129弄6号,上海市,上海,徐汇区,121.446235,31.169152,未识别,上海,未识别
1,王,上海市松江区新桥镇场南路一弄同,上海市,上海,徐汇区,121.446235,31.169152,未识别,上海,松江区
2,唐,上海市徐汇区斜土路2897弄5,上海市,上海,徐汇区,121.446235,31.169152,未识别,上海,徐汇区
3,沈,有浦东新区海阳路工闷一弄20号,上海市,上海,徐汇区,121.446235,31.169152,未识别,未识别,有浦东新区
4,顾,罗城路700弄21号302室,上海市,上海,徐汇区,121.446235,31.169152,未识别,未识别,未识别
5,王,上海中山南二路960弄1号40,上海市,上海,徐汇区,121.446235,31.169152,未识别,未识别,未识别
6,濮,上海市徐汇区桂林西街85弄68,上海市,上海,徐汇区,121.446235,31.169152,未识别,上海,徐汇区
7,张,上海市古美西路753弄31号6,上海市,上海,徐汇区,121.446235,31.169152,未识别,上海,未识别
8,郭,青岛科技大学化工学院,上海市,上海,徐汇区,121.446235,31.169152,未识别,未识别,未识别
9,陈,广州市东华东路东源新街5号80,上海市,上海,徐汇区,121.446235,31.169152,未识别,广州,未识别


In [ ]:
'''
2、查看姓氏“普遍指数”，普遍指数=姓氏人口数量
要求：
① 将数据按照“姓”做统计，找到数量最多的TOP20
② 分别制作图表，查看姓氏TOP20的数量及占比
   * 建议用bokeh出柱状图，并且为联动图表
③ 查看“王”姓的全国分布
   * 这里导出excel高版本文件，用powermap查看，绘制密度图
   * 同时可以尝试用echarts绘制空间柱状图来查看
④ 查看“姬”姓的全国分布
   * 这里导出excel高版本文件，用powermap查看，绘制密度图
   * 同时可以尝试用echarts绘制空间柱状图来查看
提示：
① bokeh中绘制联动图表时用gridplot
② powermap需要office2016的excel才会有，并且必须存储xlsx格式
③ powermap中需要通过在“值”中设置“姓的计数”才能正确显示热力图
④ powermap中可以通过“主题”来调节配色风格 / “平面地图”选项来调整球面可视化或者平面可视化
⑤ echarts绘制图表之前，需要对数据按照“lng”（或者“lat”）汇总，得到同一个地点的该姓氏人数，然后绘图
⑥ ecahrts通过设置以下参数来达到效果：视角远近、点柱大小

'''

In [5]:
# 将数据按照“姓”做统计，找到数量最多的TOP10

name_count = df['姓'].value_counts()
result1_01 = pd.DataFrame({'count':name_count, 'count_pre':name_count/name_count.sum()})
# 筛选top20的姓氏，计数并计算占比
result1_01 = result1_01[:20]
result1_01

,count,count_pre
王,135672,0.077605
张,123796,0.070812
李,112504,0.064353
刘,86352,0.049394
陈,82370,0.047116
杨,48887,0.027964
吴,36528,0.020894
赵,36467,0.020859
周,36429,0.020838
徐,34419,0.019688


In [10]:
# bokeh出联动柱状图

from bokeh.models import HoverTool
from bokeh.layouts import gridplot
# 导入模块 HoverTool悬停工具

name_lst = result1_01.index.tolist()
source = ColumnDataSource(result1_01)
# 创建ColumnDataSource数据

hover1 = HoverTool(tooltips=[("姓氏计数", "@count")])  # 设置标签显示内容
result1 = figure(plot_width=800, plot_height=250,x_range = name_lst,
                 title="中国姓氏TOP20 - 计数" ,
                 tools=[hover1,'reset,xwheel_zoom,pan,crosshair'])   # 构建绘图空间
result1.vbar(x='index', top='count', source=source,width=0.9, alpha = 0.7,color = 'red')   
result1.ygrid.grid_line_dash = [6, 4]
result1.xgrid.grid_line_dash = [6, 4]
# 柱状图1

hover2 = HoverTool(tooltips=[("姓氏占比", "@count_pre")])  # 设置标签显示内容
result2 = figure(plot_width=800, plot_height=250,x_range = result1.x_range,
                 title="中国姓氏TOP20 - 占比" ,
                 tools=[hover2,'reset,xwheel_zoom,pan,crosshair']) 
result2.vbar(x='index', top='count_pre', source=source,width=0.9, alpha = 0.7,color = 'green')   
result2.ygrid.grid_line_dash = [6, 4] # 背景网格
result2.xgrid.grid_line_dash = [6, 4] # 背景网格
# 柱状图2

p = gridplot([[result1], [result2]])
# 组合图表，网格显示
show(p) # 呈现

In [20]:
# 查看“王”姓的全国分布

data_wang1 = df[df['姓'] == '王']
writer = pd.ExcelWriter(r'C:\Users\Administrator\Python数据\网易数据分析\1项目资料\项目09中国姓氏排行研究\wang1.xlsx')
data_wang1.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据1

# data_wang2 = data_wang1.groupby(['姓','户籍所在地_lng','户籍所在地_lat'])['户籍所在地_市'].count()
data_wang2 = data_wang1.groupby(['户籍所在地_市','户籍所在地_lng','户籍所在地_lat'])['姓'].count()
data_wang2 = data_wang2.reset_index() # 该行，等于slip开了groupby的内容。
writer = pd.ExcelWriter(r'C:\Users\Administrator\Python数据\网易数据分析\1项目资料\项目09中国姓氏排行研究\wang2.xlsx')
data_wang2.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据2

print('导出完成！')
print('\ndata_wang1\n',data_wang1.head(3),'\n')
print('\ndata_wang2\n',data_wang2.head(3),'\n')
# 结论 → 老王们无处不在啊！

导出完成！

data_wang1
     姓              工作地 户籍所在地_省 户籍所在地_市 户籍所在地_区县   户籍所在地_lng  户籍所在地_lat 工作地_省  \
1   王  上海市松江区新桥镇场南路一弄同     上海市      上海      徐汇区  121.446235  31.169152   未识别   
5   王  上海中山南二路960弄1号40     上海市      上海      徐汇区  121.446235  31.169152   未识别   
17  王  徐汇区 华展路55弄 光华绿苑     上海市      上海      徐汇区  121.446235  31.169152   未识别   

   工作地_市 工作地_区县  
1     上海    松江区  
5    未识别    未识别  
17   未识别    徐汇区   


data_wang2
   户籍所在地_市   户籍所在地_lng  户籍所在地_lat   姓
0     七台河  130.818169  45.930545  64
1     七台河  130.893188  45.813494  37
2     七台河  130.992503  45.770093  51 



In [21]:
# 查看“姬”姓的全国分布

data_ji1 = df[df['姓'] == '姬']
writer = pd.ExcelWriter(r'C:\Users\Administrator\Python数据\网易数据分析\1项目资料\项目09中国姓氏排行研究\ji1.xlsx')
data_ji1.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据1

data_ji2 = data_ji1.groupby(['姓','户籍所在地_lng','户籍所在地_lat'])['户籍所在地_市'].count()
data_ji2 = data_ji2.reset_index()
writer = pd.ExcelWriter(r'C:\Users\Administrator\Python数据\网易数据分析\1项目资料\项目09中国姓氏排行研究\ji2.xlsx')
data_ji2.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据2

print('导出完成！')
print('\ndata_ji1\n',data_ji1.head(3),'\n')
print('data_ji2\n',data_ji2.head(3),'\n')
# 结论 → “姬”传说是黄帝之姓、周朝国姓，并且是10大姓中7个姓的起源
# 千年过去，姬姓后嗣多已改为他姓，开枝散叶。而还保留着这个古老姓氏的人口，也仍然栖息在古老中华文明的发源地——河南。

导出完成！

data_ji1
        姓              工作地 户籍所在地_省 户籍所在地_市 户籍所在地_区县   户籍所在地_lng  户籍所在地_lat  \
7568   姬  上海市徐汇区田林十四村35号3     上海市      上海      徐汇区  121.446235  31.169152   
22489  姬  武汉市武昌区雄楚大衔136-2     湖北省      武汉      武昌区  114.353622  30.564860   
25732  姬  江苏省南通市崇川区虹桥新村30     江苏省      南通      崇川区  120.887599  31.962661   

      工作地_省 工作地_市 工作地_区县  
7568    未识别    上海    徐汇区  
22489   未识别    武汉    武昌区  
25732    江苏    南通    崇川区   

data_ji2
    姓  户籍所在地_lng  户籍所在地_lat  户籍所在地_市
0  姬  -1.000000   -1.00000        3
1  姬  80.952156   44.96882        1
2  姬  85.709418   41.70550        1 



In [14]:
'''
3、查看姓氏“奔波指数”，奔波指数=姓氏人均迁徙距离。迁徙距离为户籍地所在地级市与现居住地所在地级市的距离。
要求：
① 根据识别的工作地，通过Geocoding查询到对应坐标
② 选择一个姓氏，计算并查看其姓氏的奔波指数，并计算该姓氏的人均通勤距离
   * 在python中筛选数据之后，qgis内做空间分析
③ 按照起点和终点做计数，汇总同一个迁徙路径的数据
④ 通过echart制作通勤OD图
   * 可以将生成的line文件导出geojson，用空间线性轨迹图来表示
   * 这里线的valye为该迁徙路径的汇总计数   
提示：
① 可以筛选一些好玩的姓氏：汤、朴、廉、何、叶、冉等等 
② 需要对数据的工作地进行筛选，其中“工作地_市”、“工作地_区县”未识别的数据删除掉
③ 导出数据时，尽量columns名用全英文，避免qgis中出现乱码
④ 计算人均通勤距离的时候，需要删除掉户籍地与工作地相同的人（未迁移的人）
⑤ 在官网metrodata.cn的小工具中找到geocoding
⑥ qgis中需要安装插件“LinePlotter”来转线，并在qgis中计算平均通勤距离（需要投影，投影经度带可选48）
⑦ shapefile转geojson时，注意shapefile文件要投影回wgs84地理坐标系

'''

In [26]:
# 计算并查看“汤”姓氏的奔波指数

data_tang = df[['姓','户籍所在地_lng','户籍所在地_lat','工作地_市','工作地_区县']][df['姓'] == '汤']
data_tang = data_tang[data_tang['工作地_市'] != '未识别']
data_tang = data_tang[data_tang['工作地_区县'] != '未识别']
data_tang.columns = ['familyname','birth_lng','birth_lat','work_city','work_district']
# 筛选并清洗数据

writer = pd.ExcelWriter(r'C:\Users\Administrator\Python数据\网易数据分析\1项目资料\项目09中国姓氏排行研究\tang.xlsx')
data_tang.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据

print('数据条数为%i条' % len(data_tang))
data_tang.head(10)

数据条数为1988条


,familyname,birth_lng,birth_lat,work_city,work_district
258,汤,121.446235,31.169152,上海,嘉定区
704,汤,121.446235,31.169152,上海,徐汇区
935,汤,121.446235,31.169152,上海,徐汇区
1215,汤,121.446235,31.169152,上海,徐汇区
1336,汤,121.446235,31.169152,上海,徐汇区
1606,汤,121.446235,31.169152,上海,徐汇区
1647,汤,121.446235,31.169152,上海,徐汇区
1810,汤,121.446235,31.169152,上海,徐汇区
2264,汤,121.446235,31.169152,上海,徐汇区
2442,汤,121.446235,31.169152,上海,徐汇区
